In [1]:
import pandas as pd
import numpy as np 


Preparing Data

In [2]:
movies = pd.read_csv('../Linear_regression/data_set/Movies/movies.csv')
ratings = pd.read_csv('../Linear_regression/data_set/Movies/ratings.csv')

In [3]:
movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Data Scraping

In [4]:
#Using regular expresion to delete year from tittle
movies['year'] = movies['title'].str.extract('(\(\d\d\d\d\))',expand = False)
movies['year'] = movies['year'].str.extract('(\d\d\d\d)',expand=False)
movies['title'] = movies['title'].str.replace('(\(\d\d\d\d\))', '')
movies['title'] = movies['title'].apply(lambda x: x.strip())
movies['genres'] = movies['genres'].str.split(pat='|')
movies.head()

C:\Users\shina\AppData\Local\Temp\ipykernel_18380\3386939449.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['title'] = movies['title'].str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [5]:
movies_hot_encode = pd.concat([             #Hot encode wykorzysytując funkcję biblioteki pandas concat oraz funkcji applay
        movies.drop("genres", axis = 1),
        movies.genres.apply(lambda x: pd.Series(1, x)).fillna(0)
    ], axis=1, )
movies_hot_encode.drop(["title","year","(no genres listed)"],axis=1,inplace=True)
movies_hot_encode.set_index(keys='movieId',drop=True,inplace=True)

In [6]:
movies_hot_encode.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir
movieId,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


User data

In [7]:
user_movies = ratings.merge(movies, left_on='movieId',right_on='movieId')#połączenie ze sobą dwóch tabel danych
user_movies.drop(['title','timestamp','genres','year'],inplace=True,axis = 1)
user_movies.set_index(keys='movieId',drop=True,inplace=True)

Recommender System

In [8]:
idx  = user_movies.loc[user_movies['userId'] == 244,:].index.tolist()#utworzenie wektora z normalizowanego do doboru polecenia filmu
value_list = movies_hot_encode.loc[idx,:]
user_ratings = user_movies.loc[user_movies['userId'] == 244,'rating'].T
genre_ratings = user_ratings.dot(value_list)
genre_ratings_normalize = (genre_ratings-genre_ratings.min())/(genre_ratings.max()-genre_ratings.min())
genre_ratings_normalize

Adventure      0.523316
Animation      0.000000
Children       0.015544
Comedy         0.404145
Fantasy        0.119171
Romance        0.295337
Drama          0.818653
Action         1.000000
Crime          0.305699
Thriller       0.756477
Horror         0.103627
Mystery        0.067358
Sci-Fi         0.326425
War            0.316062
Musical        0.000000
Documentary    0.000000
IMAX           0.000000
Western        0.067358
Film-Noir      0.015544
Name: rating, dtype: float64

In [9]:
idx1  = user_movies.loc[user_movies['userId'] != 244,:].index.tolist() # obliczenie średniej filmów
un_seen_films = movies_hot_encode.loc[np.unique(idx1),:]#użyte np.unique zamiast pd.drop_duplicates ponieważ funkcja z pandasa usuwała wartości orginalne   
ratings_un_seen_films = genre_ratings_normalize*un_seen_films
ratings_un_seen_films['Avarege']=ratings_un_seen_films.sum(axis=1)


Expected value

In [10]:
top20 = ratings_un_seen_films.loc[:,'Avarege'].sort_values(ascending = False).iloc[0:20]#wybór 20 najlepszych filmów do polecenia 
movies.loc[movies['movieId'].isin(top20.index),:]

,movieId,title,genres,year
400,459,"Getaway, The","[Action, Adventure, Crime, Drama, Romance, Thr...",1994
3460,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
3512,4800,King Solomon's Mines,"[Action, Adventure, Drama, Romance, Thriller]",1937
3608,4956,"Stunt Man, The","[Action, Adventure, Comedy, Drama, Romance, Th...",1980
4176,6016,City of God (Cidade de Deus),"[Action, Adventure, Crime, Drama, Thriller]",2002
4843,7235,Ichi the Killer (Koroshiya 1),"[Action, Comedy, Crime, Drama, Horror, Thriller]",2001
5161,8361,"Day After Tomorrow, The","[Action, Adventure, Drama, Sci-Fi, Thriller]",2004
5203,8481,Northwest Passage,"[Action, Adventure, Drama, Romance, Thriller, ...",1940
5476,26236,"White Sun of the Desert, The (Beloe solntse pu...","[Action, Adventure, Comedy, Drama, Romance, War]",1970
5665,27618,"Sound of Thunder, A","[Action, Adventure, Drama, Sci-Fi, Thriller]",2005


In [11]:
user_ratings.head(5)

movieId
6      5.0
110    5.0
151    5.0
163    3.0
316    3.0
Name: rating, dtype: float64

In [12]:
movies.title

0                                Toy Story
1                                  Jumanji
2                         Grumpier Old Men
3                        Waiting to Exhale
4              Father of the Bride Part II
                       ...                
9737    Black Butler: Book of the Atlantic
9738                 No Game No Life: Zero
9739                                 Flint
9740          Bungo Stray Dogs: Dead Apple
9741          Andrew Dice Clay: Dice Rules
Name: title, Length: 9742, dtype: object

In [13]:
import json
options = 'Toy story'
number = '5'
dict_of_watched_films ={}
dict_of_watched_films["title"]=options 
dict_of_watched_films["grade"]= number
with open("E:\Data_scientist\Linear_regression\sample.json") as file:
    data = json.load(file)
    data.append(dict_of_watched_films)
    file.seek(0)
    json.dump(data, file)


AttributeError: 'dict' object has no attribute 'append'

In [ ]:
dict_of_watched_films

{'title': 'Toy story', 'grade': '5'}

In [14]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
